In [7]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
import numpy as np
import datetime
import os

print(tf.__version__)
print(np.__version__)

2.2.0
1.18.4


In [8]:
# 在终端执行程序时指定GPU  

# CUDA_VISIBLE_DEVICES=1   python  your_file.py

# 这样在跑你的网络之前，告诉程序只能看到1号GPU，其他的GPU它不可见

# 可用的形式如下：

# CUDA_VISIBLE_DEVICES=1           Only device 1 will be seen
# CUDA_VISIBLE_DEVICES=0,1         Devices 0 and 1 will be visible
# CUDA_VISIBLE_DEVICES="0,1"       Same as above, quotation marks are optional
# CUDA_VISIBLE_DEVICES=0,2,3       Devices 0, 2, 3 will be visible; device 1 is masked
# CUDA_VISIBLE_DEVICES=""          No GPU will be visible



# 在Python代码中指定GPU

# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"



# 设置定量的GPU使用量

# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.9 # 占用GPU90%的显存
# session = tf.Session(config=config)



# 设置最小的GPU使用量

# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# session = tf.Session(config=config)

In [9]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [10]:
# tf.test.is_gpu_available() # 高版本已废弃不再使用
tf.config.list_physical_devices('GPU')

[]

In [11]:
# gpus = tf.config.list_physical_devices(device_type='GPU')
# cpus = tf.config.list_physical_devices(device_type='CPU')
# print(gpus, cpus)

[] [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [13]:
# GPU
# gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
# print(os.environ['CUDA_VISIBLE_DEVICES'])
# CPU
# cpus = tf.config.experimental.list_physical_devices(device_type='CPU')

0


In [14]:
# tf.config.list_physical_devices('GPU')

[]

In [15]:
# 划分训练数据，测试数据
mnist = np.load("mnist.npz")
x_train, y_train, x_test, y_test = mnist['x_train'],mnist['y_train'],mnist['x_test'],mnist['y_test']
# 归一化
x_train, x_test = x_train / 255.0, x_test / 255.0


In [28]:
# 增加通道维度
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

# x_train
# x_test

In [29]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [ ]:
class MyModel(Model):
    def __init__(self,**kwargs):
        super(MyModel, self).__init__(**kwargs)
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation = 'relu')
        self.d2 = Dense(10, activation='softmax')
        
    @tf.function
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)